<a href="https://colab.research.google.com/github/jaya-shankar/education-impact/blob/master/data_extract_code/edu_attainment_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!rm -rf education-impact

In [3]:
!git clone https://github.com/jaya-shankar/education-impact.git

Cloning into 'education-impact'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (341/341), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 341 (delta 170), reused 167 (delta 50), pack-reused 0
Receiving objects: 100% (341/341), 2.85 MiB | 12.52 MiB/s, done.
Resolving deltas: 100% (170/170), done.


In [5]:
import pandas as pd
import numpy as np
import csv

In [12]:
i_file_name = "education-attainment.csv"
data = pd.read_csv("education-impact/raw_data/"+i_file_name)
df = pd.DataFrame(data)
df

,Area,Year,Age,Education,Distribution
0,Afghanistan,1960,20--24,Under 15,0.0
1,Afghanistan,1965,20--24,Under 15,0.0
2,Afghanistan,1970,20--24,Under 15,0.0
3,Afghanistan,1975,20--24,Under 15,0.0
4,Afghanistan,1980,20--24,Under 15,0.0
...,...,...,...,...,...
15908,Syrian Arab Republic,2015,20--24,Master and higher,0.0
15909,Taiwan Province of China,2015,20--24,Master and higher,1.6
15910,United Kingdom of Great Britain and Northern I...,2015,20--24,Master and higher,2.3
15911,United States of America,2015,20--24,Master and higher,2.4


In [13]:
df.columns

Index(['Area', 'Year', 'Age', 'Education', 'Distribution'], dtype='object')

In [14]:
df = df[df.Education != 'Under 15']
df

,Area,Year,Age,Education,Distribution
2248,Afghanistan,1960,20--24,No Education,92.5
2249,Afghanistan,1965,20--24,No Education,90.6
2250,Afghanistan,1970,20--24,No Education,89.1
2251,Afghanistan,1975,20--24,No Education,84.2
2252,Afghanistan,1980,20--24,No Education,82.4
...,...,...,...,...,...
15908,Syrian Arab Republic,2015,20--24,Master and higher,0.0
15909,Taiwan Province of China,2015,20--24,Master and higher,1.6
15910,United Kingdom of Great Britain and Northern I...,2015,20--24,Master and higher,2.3
15911,United States of America,2015,20--24,Master and higher,2.4


In [15]:
df.Education.unique()

array(['No Education', 'Incomplete Primary', 'Primary', 'Lower Secondary',
       'Upper Secondary', 'Post Secondary', 'Short Post Secondary',
       'Bachelor', 'Master and higher'], dtype=object)

In [16]:
edu_level = { 'In_Primary_OL'     : [ 'No Education', 'Incomplete Primary'], 
              'Primary_OL'        : [ 'No Education', 'Incomplete Primary','Primary'],
              'Lower_Secondary_OL': ['No Education', 'Incomplete Primary','Primary', 'Lower Secondary'],
            }

In [17]:
for i in edu_level:
  sr = df.loc[df['Education'].isin(edu_level[i])]
  sr.to_csv('education-impact/datasets/'+i+'.csv', encoding='utf-8', index=False)

In [18]:
countries = list(df.Area.unique())

In [19]:
def init_timeline():
  timeline = [i for i in range(1960,2016)]
  timeline_dic={c:{t : 0 if not t%5 else np.NaN for t in timeline  } for c in countries}
  return timeline_dic

In [20]:
def convert_to_df():
  dft = pd.DataFrame.from_dict(timeline_dic,orient='index')
  dft.insert(0, "Country", countries, True)
  return dft

In [21]:
for e in edu_level:
  timeline_dic = init_timeline()
  data = pd.read_csv("education-impact/datasets/"+e+".csv")   
  dfs = pd.DataFrame(data)
  for j in range(len(data)):
    timeline_dic[dfs.iloc[j]['Area']][dfs.iloc[j]['Year']] += dfs.iloc[j]['Distribution']
    round(timeline_dic[dfs.iloc[j]['Area']][dfs.iloc[j]['Year']], 2)
  dft = convert_to_df()
  dft.to_csv('education-impact/datasets/'+e+'.csv', encoding='utf-8', index=False)

In [24]:
years = [(y,y+5) for y in range(1960,2011,5)]
datasets = [ e for e in edu_level]

for name in datasets:
  data = pd.read_csv('education-impact/datasets/'+name+'.csv')   
  df = pd.DataFrame(data)
  for r in range(len(df)):
    for s,e in years:
      df.at[r,str(s)] = round(df.iloc[r][str(s)],2)
      df.at[r,str(e)] = round(df.iloc[r][str(e)],2)
      d   = df.iloc[r][str(e)] - df.iloc[r][str(s)]
      inc = d/5
      i=1
      for y in range(s+1,e):
        df.at[r,str(y)] = round(df.iloc[r][str(s)] + inc*i,2)
        i+=1
  df.to_csv('education-impact/datasets/'+name+'.csv', encoding='utf-8', index=False)